In [1]:
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import mysql.connector
from flask import Flask, request, jsonify
import base64
from flask_cors import CORS
import spacy

app = Flask(__name__)
CORS(app, resources={r"/similarity/*": {"origins": "http://localhost:4200"}})

# Charger le modèle de embeddings multilingue
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Charger les modèles spaCy en français et en anglais
nlp_fr = spacy.load('fr_core_news_sm')
nlp_en = spacy.load('en_core_web_sm')

# Liste des diplômes
DEGREES = ["licence", "ingenieur", "master","ingénieur"]

# Fonction pour extraire les diplômes d'un texte de CV
def extract_degree(resume_text):
    nlp_text = nlp_fr(resume_text)
    degrees = []
    for token in nlp_text:
        if token.text.lower() in DEGREES:
            degrees.append(token.text)
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in DEGREES:
            degrees.append(chunk.text)
    return [word.capitalize() for word in set(degrees)]

# Charger la base de données des compétences à partir d'un fichier externe
with open('linkedin') as f:
    external_source = list(f)

result = [element.strip().lower() for element in external_source]

# Fonction pour extraire les compétences d'un texte de CV
def extract_skill(resume_text):
    nlp_text = nlp_en(resume_text)
    tokens = [token.text for token in nlp_text if not token.is_stop]
    skills = result
    skillset = []
    for i in tokens:
        if i.lower() in skills:
            skillset.append(i)
    for i in nlp_text.noun_chunks:
        i = i.text.lower().strip()
        if i in skills:
            skillset.append(i)
    return [word.capitalize() for word in set([word.lower() for word in skillset])]

# Connexion à la base de données MySQL
def connect_db():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='ichrak'
    )

# Fonction pour extraire le texte d'un PDF stocké en BLOB
def extract_text_from_pdf(pdf_blob):
    pdf_data = fitz.open(stream=pdf_blob, filetype="pdf")
    text = ""
    for page_num in range(len(pdf_data)):
        page = pdf_data.load_page(page_num)
        text += page.get_text()
    return text

# Fonction pour récupérer les informations sur un job par son ID
def get_job_by_id(cursor, job_id):
    cursor.execute("SELECT competences, diplome FROM jobs WHERE id = %s", (job_id,))
    job = cursor.fetchone()
    return job

# Fonction pour récupérer les CVs pour une offre d'emploi spécifique
def get_cvs_by_job_id(cursor, job_id):
    cursor.execute("SELECT id, cv FROM postulation WHERE idj = %s", (job_id,))
    cvs = cursor.fetchall()
    return cvs

# Fonction pour récupérer les détails des postulations par leurs IDs dans le bon ordre
def get_postulations_by_ids(cursor, postulation_ids):
    if not postulation_ids:
        return []

    format_strings = ','.join(['%s'] * len(postulation_ids))
    cursor.execute(f"""
        SELECT 
            id, status, civilite, competences, cv, diplomes, domaine, experience_prof, 
            idc, idj, prenom, region, tel, nom, scors, horaire, date, 
            meet, date_publication, idcondidature, score
        FROM postulation WHERE id IN ({format_strings})
        ORDER BY FIELD(id, {format_strings})
    """, tuple(postulation_ids) + tuple(postulation_ids))
    postulations = cursor.fetchall()
    return postulations

@app.route('/similarity/<int:job_id>', methods=['GET'])
def calculate_similarity(job_id):
    db_conn = connect_db()
    cursor = db_conn.cursor()
    
    job = get_job_by_id(cursor, job_id)
    
    if not job:
        cursor.close()
        db_conn.close()
        return jsonify({'message': 'Job not found'}), 404
    
    # Générer les embeddings pour la description du job
    job_text = " ".join(job)  # Concatène les champs 'competences' et 'diplome'
    job_embedding = model.encode([job_text])
    
    # Récupérer les CVs pour cette offre d'emploi
    cvs = get_cvs_by_job_id(cursor, job_id)
    
    if not cvs:
        cursor.close()
        db_conn.close()
        return jsonify({'message': 'No CVs found for this job'}), 404
    
    cv_texts = []
    for cv in cvs:
        cv_id, cv_blob = cv
        cv_text = extract_text_from_pdf(cv_blob)
        print(cv_text)
        cv_degrees = extract_degree(cv_text)
        print("cv_degrees",cv_degrees)
    
        cv_skills = extract_skill(cv_text)
        print(cv_skills)
        cv_combined_text = " ".join(cv_degrees + cv_skills)
        cv_texts.append((cv_id, cv_combined_text))
    
    # Générer les embeddings pour les CVs
    cv_embeddings = model.encode([cv_text for _, cv_text in cv_texts])
    
    # Convertir les embeddings en matrices NumPy
    job_embedding = np.array(job_embedding)
    cv_embeddings = np.array(cv_embeddings)
    
    # Utiliser FAISS pour créer une instance de recherche de vecteurs pour les CVs
    index = faiss.IndexFlatL2(cv_embeddings.shape[1])
    index.add(cv_embeddings)
    
    # Calculer la similarité des CVs avec le job
    D, I = index.search(job_embedding, len(cv_embeddings))  # Trouver la similarité avec l'offre d'emploi
    
    # Afficher les IDs des CVs et leurs distances dans la console
    for idx, distance in zip(I[0], D[0]):
        print(f"CV ID: {cvs[idx][0]}, Distance: {distance}")
    
    # Trier les résultats par similarité
    results = sorted(zip(I[0], D[0]), key=lambda x: x[1])
    
    sorted_cv_ids = [cvs[idx][0] for idx, _ in results]
    
    # Récupérer les détails des postulations triées par similarité
    sorted_postulations = get_postulations_by_ids(cursor, sorted_cv_ids)
    
    # Convertir les résultats en un format sérialisable en JSON
    serialized_postulations = []
    for p in sorted_postulations:
        serialized_postulations.append({
            'id': p[0], 'status': p[1], 'civilite': p[2], 'competences': p[3],
            'cv': base64.b64encode(p[4]).decode('utf-8'),  # Encodage du CV en base64
            'diplomes': p[5], 'domaine': p[6], 'experienceProf': p[7], 'idc': p[8], 'idj': p[9],
            'prenom': p[10], 'region': p[11], 'tel': p[12], 'nom': p[13], 'scors': p[14],
            'horaire': p[15], 'date': p[16], 'meet': p[17], 
            'date_publication': p[18], 'idcondidature': p[19], 'score': p[20]
        })
    
    cursor.close()
    db_conn.close()
    return jsonify(serialized_postulations), 200

if __name__ == '__main__':
    app.run()


c:\Users\Yassine\anaconda3\envs\pfe\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1 
 
                                                                                                                                                               
                                      
 
 
 
 
 
 
 
 
 
Ingénieur en Génie    
mécanique 
 
 
 
 
De Janvier 2017 jusqu’à maintenant : Ingénieur mécanique au sein de la société étatique SEPJ Gabes 
                           Maintenance mécanique et gestion d’équipe. 
De septembre 2015 au juin 2016 : Ingénieur mécanique au sein de CLEVERTEK PLUS 
                          Conception et fabrication mécanique : utilisation des logiciels SOLIDWORKS et 
MASTERCAM pour la CAO et la FAO   
                          Usinage numérique : exploitation de trois machines à trois axes (cincinatti, bridgeport et 
rosilio),  une machine DMG à cinq axes, et une tour numérique FAGOR 8050 
                          Etude des projets mécaniques : préparation de la conception d’un mécanisme ou une pièce 
unitaire conformément aux cahiers 

127.0.0.1 - - [06/Jun/2024 12:34:33] "GET /similarity/652 HTTP/1.1" 200 -


...

['Php', 'Groups', 'C', 'Des', 'Java', 'Mes', 'Au', 'Cloud', 'Spring', 'Web', 'Python', 'Mail', 'Firebase', 'Learning', 'Mobile', 'Css', 'Security', 'Unity', 'Maps', 'Linux', 'Django', 'Application', 'Docker', 'Coursera', 'Mining', 'Vhdl', 'Consultations', 'Google', 'C++', 'Intelligence', 'Mysql', 'Html', 'Sql', 'Chat', 'Angular', 'Api', 'Nlp', 'Deep learning', 'Mongodb', 'Boot', 'R']
CV ID: 73, Distance: 1.2221665382385254
CV ID: 72, Distance: 1.5525627136230469
1 
 
                                                                                                                                                               
                                      
 
 
 
 
 
 
 
 
 
Ingénieur en Génie    
mécanique 
 
 
 
 
De Janvier 2017 jusqu’à maintenant : Ingénieur mécanique au sein de la société étatique SEPJ Gabes 
                           Maintenance mécanique et gestion d’équipe. 
De septembre 2015 au juin 2016 : Ingénieur mécanique au sein de CLEVERTEK PLUS 
                 

127.0.0.1 - - [06/Jun/2024 12:39:31] "GET /similarity/652 HTTP/1.1" 200 -


['Php', 'Groups', 'C', 'Des', 'Java', 'Mes', 'Au', 'Cloud', 'Spring', 'Web', 'Python', 'Mail', 'Firebase', 'Learning', 'Mobile', 'Css', 'Security', 'Unity', 'Maps', 'Linux', 'Django', 'Application', 'Docker', 'Coursera', 'Mining', 'Vhdl', 'Consultations', 'Google', 'C++', 'Intelligence', 'Mysql', 'Html', 'Sql', 'Chat', 'Angular', 'Api', 'Nlp', 'Deep learning', 'Mongodb', 'Boot', 'R']
CV ID: 73, Distance: 1.2221665382385254
CV ID: 72, Distance: 1.5525627136230469
1 
 
                                                                                                                                                               
                                      
 
 
 
 
 
 
 
 
 
Ingénieur en Génie    
mécanique 
 
 
 
 
De Janvier 2017 jusqu’à maintenant : Ingénieur mécanique au sein de la société étatique SEPJ Gabes 
                           Maintenance mécanique et gestion d’équipe. 
De septembre 2015 au juin 2016 : Ingénieur mécanique au sein de CLEVERTEK PLUS 
                 

127.0.0.1 - - [06/Jun/2024 12:39:32] "GET /similarity/652 HTTP/1.1" 200 -


['Php', 'Groups', 'C', 'Des', 'Java', 'Mes', 'Au', 'Cloud', 'Spring', 'Web', 'Python', 'Mail', 'Firebase', 'Learning', 'Mobile', 'Css', 'Security', 'Unity', 'Maps', 'Linux', 'Django', 'Application', 'Docker', 'Coursera', 'Mining', 'Vhdl', 'Consultations', 'Google', 'C++', 'Intelligence', 'Mysql', 'Html', 'Sql', 'Chat', 'Angular', 'Api', 'Nlp', 'Deep learning', 'Mongodb', 'Boot', 'R']
CV ID: 73, Distance: 1.2221665382385254
CV ID: 72, Distance: 1.5525627136230469


In [12]:
DEGREES = ["licence", "ingenieur", "master"]

# Fonction pour extraire les diplômes d'un texte de CV
def extract_degree(resume_text):
    nlp_text = nlp_fr(resume_text)
    degrees = []
    for token in nlp_text:
        if token.text.lower() in DEGREES:
            degrees.append(token.text)
    for chunk in nlp_text.noun_chunks:
        chunk_text = chunk.text.lower().strip()
        if chunk_text in DEGREES:
            degrees.append(chunk.text)
    return [word.capitalize() for word in set(degrees)]
 extract_degree("licence")

['Ingénieur']

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')


In [5]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

  You can safely remove it manually.



   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.2 MB 812.7 kB/s eta 0:00:15
   ---------------------------------------- 0.1/12.2 MB 837.8 kB/s eta 0:00:15
    --------------------------------------- 0.2/12.2 MB 1.2 MB/s eta 0:00:11
    --------------------------------------- 0.2/12.2 MB 1.0 MB/s eta 0:00:12
   - -------------------------------------- 0.3/12.2 MB 1.3 MB/s eta 0:00:10
   - -------------------------------------- 0.3/12.2 MB 1.4 MB/s eta 0:00:09
   - -------------------------------------- 0.5/12.2 MB 1.6 MB/s eta 0:00:08
   -- ------------------------------------- 0.7/12.2 MB 1.8 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/12.2 MB 2.1 MB/s eta 0:00:06
   --- ------------------------------------ 1.1/12.2 MB 2.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.2/12.2 MB 2.5 MB/s eta 0:00:05
   ---- 

In [21]:
! pip install spacy sentence-transformers  numpy mysql-connector-python flask flask-cors pymupdf


In [16]:
import re

def extract_profile(cv_text):
    # Utiliser une expression régulière pour rechercher la partie "PROFILE"
    pattern = re.compile(r"(?i)\bPROFILE\b:(.*?)\n\n", re.DOTALL)
    match = re.search(pattern, cv_text)

    # Si une correspondance est trouvée, retourner le contenu du profil
    if match:
        return match.group(1).strip()
    else:
        return None

def test_extract_profile():
    # Exemple de CV
    cv_text = """
    PROFILE:
    Dynamic and motivated professional with a proven record of generating and
    building relationships, managing projects from concept to completion,
    designing educational strategies, and coaching individuals to success. 
    """

    # Appel de la fonction pour extraire la partie "PROFILE"
    profile = extract_profile(cv_text)

    # Affichage du résultat du test
    print("Profile extracted from CV:")
    print(profile)

# Appel de la fonction de test
test_extract_profile()


Profile extracted from CV:
None


In [9]:
! pip install spacy

In [2]:
from flask import Flask, request, jsonify, send_file
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
from gtts import gTTS
import re

app = Flask(__name__)

# Charger le modèle de embeddings multilingue
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Exemple de textes en arabe
texts = [
    """الوثائق المطلوبة لإستخراج بطاقة تعريف وطنية لأول مرة :مطبوعة إدارية للتعمير.مضمون ولادة مستخرج من السجلات الأصلية للحالة المدنية لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
شهادة في الجنسية التونسية- شهادة إقامة
شهادة عمل أو شهادة حضور مدرسي أو جامعي
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
وصل خلاص تسلمه قباضات المالية قيمته 3 دنانير
نسخة مصورة من بطاقة إقامة ونسخة من بطاقة القنصلية بالنسبة للمقيمين بالخارج(في نظيرين)
ترخيص من الولي الشرعي معرف بالإمضاء ومعلل في الغرض لإستخراج البطاقة : (مهني، تربوي، رياضي أو بدني) بالنسبة للقصر دون 18 سنة
شهادة في الفصيلة الدموية (اختيارية)""",
    """الوثائق مطلوبة إستبدال بطاقة تعريف الوطنية هي
تعمير المطبوعة الإدارية وإمضاؤهامضمون ولادة لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
نسخة مصورة من البطاقة المطلوب تعويضها مع الاستظهار بالأصل الذي يحتفظ به المعني على أن يسلمه للمصالح المختصة عند تسلم البطاقة الجديدة
وثيقة مثبتة لتغيير عناصر الحالة المدنية أو المقر أو المهنة (عند الاقتضاء)
شهادة حضور بالنسبة للطلبة لم يمضي على تاريخ إستلامها ثلاثة أشهر
وصل خلاص تسلمه قباضات المالية""",
    """الوثائق المطلوبة للإستخراج جواز سفر هي:
تعمير إستمارة الحصول على جواز سفر عادي مقروء آليا وإمضاؤها بصفة شخصية داخل الخانة المعدة للغرض
نسخة من بطاقة التعريف الوطنية مع الاستظهار بالأصل أو مضمون ولادة بالنسبة للقصر
04 صور شمسية حسب المواصفات التالية: أن تكون خلفية الصورة بيضاء أن يكون حجم الصورة 3.5/4.5 صم تقريبا
ما يفيد الدراسة بالنسبة للتلاميذ والطلبة
ترخيص الولي بالنسبة للقصر مصحوبا بنسخة من بطاقة تعريف
وصل خلاص تسلمه قباضات المالية قيمته:25 دنانيرا بالنسبة للطلبة والتلاميذ الذين أثبتوا صفتهم تلك بتقديم شهادة أو الأطفال الذين لم يبلغوا سنّ السابعة وكذلك التمديد في صلوحيتها
80 دينارا بالنسبة للبقية وكذلك التمديد في صلوحيتها
إضافة الجواز القديم في حالة تجديد
تقديم طلب على ورق عادي عند الرغبة في الإحتفاظ بالجواز القديم""",
    """فتح حساب إدخار :مطبوعة فتح حساب متوفرة في البريد،نسخة من بطاقة تعريف،المبلغ المالي الذي سيتم إدخاره
فتح حساب جاري:شهادة عمل أصلية وليست نسخة،نسخة من بطاقة تعريف الوطنية،مطبوعة فتح حساب جاري ومطبوعة مصاحبه لها (متوفر في البريد)،10د معلوم الخدمة"""
]

# Générer les embeddings pour les textes
embeddings = model.encode(texts)

# Convertir les embeddings en une matrice NumPy
embeddings = np.array(embeddings)

# Utiliser FAISS pour créer une instance de recherche de vecteurs
index = faiss.IndexFlatL2(embeddings.shape[1])  # Dimension des embeddings
index.add(embeddings)

# Mots significatifs en arabe
significant_words = ["سفر", "جواز", "بطاقة تعريف", "بطاقة تعريف وطنية", "لإستبدال", "جواز سفر", "حساب إدخار", "حساب", "إدخار", "إستبدال"]

# Fonction pour vérifier si le texte contient des caractères arabes
def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))

# Fonction pour nettoyer le texte et vérifier les caractères valides
def clean_text(text):
    # Supprimer les symboles non désirés et conserver les caractères arabes
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    return cleaned_text

# Fonction pour vérifier si le texte contient des mots significatifs
def contains_significant_words(text, words):
    for word in words:
        if word in text:
            return True
    return False

# Fonction pour rechercher un document similaire
def search_document(query, k=1):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)  # k pour trouver les k voisins les plus proches
    return [(texts[i], D[0][j]) for j, i in enumerate(I[0])]

@app.route('/search', methods=['POST'])
def search():
    data = request.get_json()
    query = data.get('question')
    
    if not query:
        return jsonify({'message': 'Question is required'}), 400
    
    # Vérifier si la question contient des caractères arabes
    if not is_arabic(query):
        return jsonify({'message': 'Question must be in Arabic'}), 400
    
    # Nettoyer le texte de la question
    cleaned_query = clean_text(query)
    
    # Vérifier si la question contient des mots significatifs
    if not contains_significant_words(cleaned_query, significant_words):
        return jsonify({'message': 'Question must contain significant Arabic words'}), 400
    
    # Rechercher le document le plus proche
    results = search_document(cleaned_query, k=1)
    
    if results:
        best_result = results[0]
        similarity_score = best_result[1]
        # Ajuster le seuil de similarité ici (par exemple, 0.8)
        if similarity_score < 10:
            return jsonify({'repense': best_result[0]}), 200
        else:
            return jsonify({'message': 'لم يتم العثور على إجابة.'}), 404
    else:
        return jsonify({'message': 'لم يتم العثور على إجابة.'}), 404

@app.route('/text-to-audio', methods=['POST'])
def text_to_audio():
    data = request.get_json()
    text = data.get('text')
    
    if not text:
        return jsonify({'message': 'Text is required'}), 400
    
    # Convertir le texte en audio
    tts = gTTS(text, lang='ar')
    audio_file = "audio_output.mp3"
    tts.save(audio_file)
    
    return send_file(audio_file, mimetype='audio/mpeg')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2024 14:31:20] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 14:31:34] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 14:32:46] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 14:32:47] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 14:34:01] "POST /search HTTP/1.1" 400 -
127.0.0.1 - - [05/Jun/2024 14:34:12] "POST /search HTTP/1.1" 400 -


In [3]:
from flask import Flask, request, jsonify, send_file
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
from gtts import gTTS
import re

app = Flask(__name__)

# Charger le modèle de embeddings multilingue
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Exemple de textes en arabe
texts = [
    """الوثائق المطلوبة لإستخراج بطاقة تعريف وطنية لأول مرة :مطبوعة إدارية للتعمير.مضمون ولادة مستخرج من السجلات الأصلية للحالة المدنية لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
شهادة في الجنسية التونسية- شهادة إقامة
شهادة عمل أو شهادة حضور مدرسي أو جامعي
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
وصل خلاص تسلمه قباضات المالية قيمته 3 دنانير
نسخة مصورة من بطاقة إقامة ونسخة من بطاقة القنصلية بالنسبة للمقيمين بالخارج(في نظيرين)
ترخيص من الولي الشرعي معرف بالإمضاء ومعلل في الغرض لإستخراج البطاقة : (مهني، تربوي، رياضي أو بدني) بالنسبة للقصر دون 18 سنة
شهادة في الفصيلة الدموية (اختيارية)""",
    """الوثائق مطلوبة لإستبدال بطاقة تعريف الوطنية هي
تعمير المطبوعة الإدارية وإمضاؤهامضمون ولادة لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
نسخة مصورة من البطاقة المطلوب تعويضها مع الاستظهار بالأصل الذي يحتفظ به المعني على أن يسلمه للمصالح المختصة عند تسلم البطاقة الجديدة
وثيقة مثبتة لتغيير عناصر الحالة المدنية أو المقر أو المهنة (عند الاقتضاء)
شهادة حضور بالنسبة للطلبة لم يمضي على تاريخ إستلامها ثلاثة أشهر
وصل خلاص تسلمه قباضات المالية""",
    """الوثائق المطلوبة للإستخراج جواز سفر هي:
تعمير إستمارة الحصول على جواز سفر عادي مقروء آليا وإمضاؤها بصفة شخصية داخل الخانة المعدة للغرض
نسخة من بطاقة التعريف الوطنية مع الاستظهار بالأصل أو مضمون ولادة بالنسبة للقصر
04 صور شمسية حسب المواصفات التالية: أن تكون خلفية الصورة بيضاء أن يكون حجم الصورة 3.5/4.5 صم تقريبا
ما يفيد الدراسة بالنسبة للتلاميذ والطلبة
ترخيص الولي بالنسبة للقصر مصحوبا بنسخة من بطاقة تعريف
وصل خلاص تسلمه قباضات المالية قيمته:25 دنانيرا بالنسبة للطلبة والتلاميذ الذين أثبتوا صفتهم تلك بتقديم شهادة أو الأطفال الذين لم يبلغوا سنّ السابعة وكذلك التمديد في صلوحيتها
80 دينارا بالنسبة للبقية وكذلك التمديد في صلوحيتها
إضافة الجواز القديم في حالة تجديد
تقديم طلب على ورق عادي عند الرغبة في الإحتفاظ بالجواز القديم""",
    """فتح حساب إدخار :مطبوعة فتح حساب متوفرة في البريد،نسخة من بطاقة تعريف،المبلغ المالي الذي سيتم إدخاره
فتح حساب جاري:شهادة عمل أصلية وليست نسخة،نسخة من بطاقة تعريف الوطنية،مطبوعة فتح حساب جاري ومطبوعة مصاحبه لها (متوفر في البريد)،10د معلوم الخدمة"""
]

# Générer les embeddings pour les textes
embeddings = model.encode(texts)

# Convertir les embeddings en une matrice NumPy
embeddings = np.array(embeddings)

# Utiliser FAISS pour créer une instance de recherche de vecteurs
index = faiss.IndexFlatL2(embeddings.shape[1])  # Dimension des embeddings
index.add(embeddings)

# Mots significatifs en arabe
significant_words = ["سفر", "جواز", "بطاقة تعريف", "بطاقة تعريف وطنية", "لإستبدال", "جواز سفر", "حساب إدخار", "حساب", "إدخار", "إستبدال"]

# Fonction pour vérifier si le texte contient des caractères arabes
def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))

# Fonction pour nettoyer le texte et vérifier les caractères valides
def clean_text(text):
    # Supprimer les symboles non désirés et conserver les caractères arabes
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    return cleaned_text

# Fonction pour vérifier si le texte contient des mots significatifs
def contains_significant_words(text, words):
    for word in words:
        if word in text:
            return True
    return False

# Fonction pour rechercher un document similaire
def search_document(query, k=1):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)  # k pour trouver les k voisins les plus proches
    return [(texts[i], D[0][j]) for j, i in enumerate(I[0])]

@app.route('/search', methods=['POST'])
def search():
    data = request.get_json()
    query = data.get('question')
    
    if not query:
        return jsonify({'message': 'Question is required'}), 400
    
    # Vérifier si la question contient des caractères arabes
    if not is_arabic(query):
        return jsonify({'message': 'Question must be in Arabic'}), 400
    
    # Nettoyer le texte de la question
    cleaned_query = clean_text(query)
    
    # Vérifier si la question contient des mots significatifs
    if not contains_significant_words(cleaned_query, significant_words):
        return jsonify({'message': 'Question must contain significant Arabic words'}), 400
    
    # Rechercher le document le plus proche
    results = search_document(cleaned_query, k=1)
    
    if results:
        best_result = results[0]
        similarity_score = best_result[1]
        # Ajuster le seuil de similarité ici
        similarity_threshold = 0.3  # Vous pouvez ajuster ce seuil selon vos besoins
        if similarity_score < similarity_threshold:
            return jsonify({'repense': best_result[0]}), 200
        else:
            return jsonify({'message': 'لم يتم العثور على إجابة.'}), 404
    else:
        return jsonify({'message': 'لم يتم العثور على إجابة.'}), 404

@app.route('/text-to-audio', methods=['POST'])
def text_to_audio():
    data = request.get_json()
    text = data.get('text')
    
    if not text:
        return jsonify({'message': 'Text is required'}), 400
    
    # Convertir le texte en audio
    tts = gTTS(text, lang='ar')
    audio_file = "audio_output.mp3"
    tts.save(audio_file)
    
    return send_file(audio_file, mimetype='audio/mpeg')

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2024 14:35:31] "POST /search HTTP/1.1" 400 -
127.0.0.1 - - [05/Jun/2024 14:35:32] "POST /search HTTP/1.1" 400 -
127.0.0.1 - - [05/Jun/2024 14:35:39] "POST /search HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2024 14:36:20] "POST /search HTTP/1.1" 404 -
127.0.0.1 - - [05/Jun/2024 14:40:52] "POST /search HTTP/1.1" 404 -


In [ ]:
from flask import Flask, request, jsonify, send_file
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
from gtts import gTTS
import re

app = Flask(__name__)

# Charger le modèle de embeddings multilingue
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Exemple de textes en arabe
texts = [
    """الوثائق المطلوبة لإستخراج بطاقة تعريف وطنية لأول مرة :مطبوعة إدارية للتعمير.مضمون ولادة مستخرج من السجلات الأصلية للحالة المدنية لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
شهادة في الجنسية التونسية- شهادة إقامة
شهادة عمل أو شهادة حضور مدرسي أو جامعي
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
وصل خلاص تسلمه قباضات المالية قيمته 3 دنانير
نسخة مصورة من بطاقة إقامة ونسخة من بطاقة القنصلية بالنسبة للمقيمين بالخارج(في نظيرين)
ترخيص من الولي الشرعي معرف بالإمضاء ومعلل في الغرض لإستخراج البطاقة : (مهني، تربوي، رياضي أو بدني) بالنسبة للقصر دون 18 سنة
شهادة في الفصيلة الدموية (اختيارية)""",
    """الوثائق مطلوبة لإستبدال بطاقة تعريف الوطنية هي
تعمير المطبوعة الإدارية وإمضاؤهامضمون ولادة لم يمض على تاريخ إصداره أكثر من ثلاثة أشهر
ثلاثة صور فوتوغرافية للمعني بالأمر تكون من حجم 4/3 صم وتؤخذ وجها على لوحة خلفية من اللون الأبيض أو الفاتح وبمقياس 10/1 وتبين الشعر والعينين
نسخة مصورة من البطاقة المطلوب تعويضها مع الاستظهار بالأصل الذي يحتفظ به المعني على أن يسلمه للمصالح المختصة عند تسلم البطاقة الجديدة
وثيقة مثبتة لتغيير عناصر الحالة المدنية أو المقر أو المهنة (عند الاقتضاء)
شهادة حضور بالنسبة للطلبة لم يمضي على تاريخ إستلامها ثلاثة أشهر
وصل خلاص تسلمه قباضات المالية""",
    """الوثائق المطلوبة للإستخراج جواز سفر هي:
تعمير إستمارة الحصول على جواز سفر عادي مقروء آليا وإمضاؤها بصفة شخصية داخل الخانة المعدة للغرض
نسخة من بطاقة التعريف الوطنية مع الاستظهار بالأصل أو مضمون ولادة بالنسبة للقصر
04 صور شمسية حسب المواصفات التالية: أن تكون خلفية الصورة بيضاء أن يكون حجم الصورة 3.5/4.5 صم تقريبا
ما يفيد الدراسة بالنسبة للتلاميذ والطلبة
ترخيص الولي بالنسبة للقصر مصحوبا بنسخة من بطاقة تعريف
وصل خلاص تسلمه قباضات المالية قيمته:25 دنانيرا بالنسبة للطلبة والتلاميذ الذين أثبتوا صفتهم تلك بتقديم شهادة أو الأطفال الذين لم يبلغوا سنّ السابعة وكذلك التمديد في صلوحيتها
80 دينارا بالنسبة للبقية وكذلك التمديد في صلوحيتها
إضافة الجواز القديم في حالة تجديد
تقديم طلب على ورق عادي عند الرغبة في الإحتفاظ بالجواز القديم""",
    """فتح حساب إدخار :مطبوعة فتح حساب متوفرة في البريد،نسخة من بطاقة تعريف،المبلغ المالي الذي سيتم إدخاره
فتح حساب جاري:شهادة عمل أصلية وليست نسخة،نسخة من بطاقة تعريف الوطنية،مطبوعة فتح حساب جاري ومطبوعة مصاحبه لها (متوفر في البريد)،10د معلوم الخدمة"""
]

# Générer les embeddings pour les textes
embeddings = model.encode(texts)

# Convertir les embeddings en une matrice NumPy
embeddings = np.array(embeddings)

# Utiliser FAISS pour créer une instance de recherche de vecteurs
index = faiss.IndexFlatL2(embeddings.shape[1])  # Dimension des embeddings
index.add(embeddings)

# Mots significatifs en arabe
significant_words = ["سفر", "جواز", "بطاقة تعريف", "بطاقة تعريف وطنية", "إستبدال", "جواز سفر", "حساب إدخار", "حساب", "إدخار"]

# Fonction pour vérifier si le texte contient des caractères arabes
def is_arabic(text):
    return bool(re.search(r'[\u0600-\u06FF]', text))

# Fonction pour nettoyer le texte et vérifier les caractères valides
def clean_text(text):
    # Supprimer les symboles non désirés et conserver les caractères arabes
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Supprimer les mots d'une longueur inférieure à 2 caractères
    cleaned_text = ' '.join([word for word in cleaned_text.split() if len(word) > 1])
    return cleaned_text

# Fonction pour normaliser les mots en supprimant les préfixes courants
def normalize_text(text):
    prefixes = ['ل', 'ب', 'و', 'ف', 'ك', 'س']
    words = text.split()
    normalized_words = []
    for word in words:
        if any(word.startswith(prefix) for prefix in prefixes):
            for prefix in prefixes:
                if word.startswith(prefix):
                    word = word[len(prefix):]
                    break
        normalized_words.append(word)
    return ' '.join(normalized_words)

# Fonction pour vérifier si le texte contient des mots significatifs
def contains_significant_words(text, words):
    normalized_text = normalize_text(text)
    for word in words:
        if word in normalized_text:
            return True
    return False

# Fonction pour rechercher un document similaire
def search_document(query, k=1):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)  # k pour trouver les k voisins les plus proches
    return [(texts[i], D[0][j]) for j, i in enumerate(I[0])]

@app.route('/search', methods=['POST'])
def search():
    data = request.get_json()
    query = data.get('question')
    
    if not query:
        return jsonify({'message': 'Question is required'}), 400
    
    # Vérifier si la question contient des caractères arabes
    if not is_arabic(query):
        return jsonify({'message': 'Question must be in Arabic'}), 400
    
    # Nettoyer le texte de la question
    cleaned_query = clean_text(query)
    
    # Vérifier si la question contient des mots significatifs
    if not contains_significant_words(cleaned_query, significant_words):
        return jsonify({'message': 'Question must contain significant Arabic words'}), 400
    
    # Rechercher le document le plus proche
    results = search_document(cleaned_query, k=1)
    
    if results:
        best_result = results[0]
        return jsonify({'repense': best_result[0]}), 200
    else:
        return jsonify({'message': 'لم يتم العثور على إجابة.'}), 404

@app.route('/text-to-audio', methods=['POST'])
def text_to_audio():
    data = request.get_json()
    text = data.get('text')
    
    if not text:
        return jsonify({'message': 'Text is required'}), 400
    
    # Convertir le texte en audio
    tts = gTTS(text, lang='ar')
    audio_file = "audio_output.mp3"
    tts.save(audio_file)
    
    return send_file(audio_file, mimetype='audio/mpeg')

if __name__ == '__main__':
    app.run()


c:\Users\Yassine\anaconda3\envs\pfe\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jun/2024 15:06:31] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [05/Jun/2024 15:06:36] "POST /search HTTP/1.1" 200 -


In [4]:
import os
import subprocess

def get_package_size(package_name):
    try:
        result = subprocess.run(['pip', 'show', package_name], capture_output=True, text=True)
        location = None
        for line in result.stdout.split('\n'):
            if line.startswith('Location:'):
                location = line.split(': ')[1]
        if location:
            package_path = os.path.join(location, package_name)
            size = sum(os.path.getsize(os.path.join(dirpath, filename))
                       for dirpath, dirnames, filenames in os.walk(package_path)
                       for filename in filenames)
            return size
        else:
            return 0
    except Exception as e:
        return 0

packages = ['flask', 'sentence-transformers', 'faiss', 'numpy', 'gtts', 're']

total_size = 0
for package in packages:
    size = get_package_size(package)
    print(f'Package: {package}, Size: {size / (1024 * 1024):.2f} MB')
    total_size += size

print(f'Total size: {total_size / (1024 * 1024):.2f} MB')


Package: flask, Size: 0.59 MB
Package: sentence-transformers, Size: 0.00 MB
Package: faiss, Size: 0.00 MB
Package: numpy, Size: 59.23 MB
Package: gtts, Size: 0.12 MB
Package: re, Size: 0.00 MB
Total size: 59.94 MB


In [3]:
pip install langid

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB 653.6 kB/s eta 0:00:03
     - -------------------------------------- 0.1/1.9 MB 825.8 kB/s eta 0:00:03
     - -------------------------------------- 0.1/1.9 MB 825.8 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.9 MB 774.0 kB/s eta 0:00:03
     --- ------------------------------------ 0.2/1.9 MB 701.4 kB/s eta 0:00:03
     ----- ---------------------------------- 0.3/1.9 MB 1.1 MB/s eta 0:00:02
     --------- ------------------------------ 0.5/1.9 MB 1.6 MB/s eta 0:00:01
     ------------ 

In [24]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.
